In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.metrics import classification_report
import time
import numpy as np
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
# pd.options.display.max_columns = None

predictor = TabularPredictor.load('./AutogluonModels/final', require_version_match=False)

# print(f"\n\n training report: ")
# predictor.fit_summary()
# 
# print(f"\n\n validation leaderboard: ")
# predictor.leaderboard()

# test_data = pd.read_csv('./AutogluonModels/test_data_gluon.csv')
test_data = TabularDataset('test_data_gluon.csv')
test_data.drop(columns='Unnamed: 0', inplace=True)

train_data = TabularDataset('test_data_gluon.csv')
train_data.drop(columns='Unnamed: 0', inplace=True)

# y_pred = predictor.predict(test_data)

# print("\nEvaluation_with sklearn:")
# t = time.process_time()
# y_pred = predictor.predict(test_data, model='LightGBM_BAG_L1')
# cls_report = classification_report(test_data.Efectividad.values, y_pred, output_dict=True)
# elapsed_time = time.process_time() - t
# print(cls_report)

test_acc = predictor.evaluate(test_data, model='NeuralNetTorch_DSTL')['accuracy']
print('\ntest_accuracy: ', test_acc)

y_pred = predictor.predict_proba(test_data, model='NeuralNetTorch_DSTL', as_pandas=True, as_multiclass=True)

# print("\ny_pred: \n", y_pred)


def get_simple_window(train_data, test_data):
    """
    get a dicto with max-min values.
    What if the values have some discontinuities?
    show max min or even show a map
    """
    # print(test_df.values)
    feature_generator = AutoMLPipelineFeatureGenerator()
    X_train = train_data.drop(labels=['Efectividad'], axis=1)
    y_train = train_data['Efectividad']
    X_train_transformed = feature_generator.fit_transform(X=X_train, y=y_train)
    X_test_transformed = feature_generator.transform(test_data)
    print('\nX_test_transformed: \n', list(X_test_transformed.columns))
    d = X_test_transformed.describe()
    print("\ndescribe rows: ", list(d.index))
    return d.loc[['min', '25%','50%','75%', 'max']]

test_df_summary = get_simple_window(train_data, test_data)


############################## WARNING ##############################
	Predictor Version: 0.4.1b20220430
	Current Version:   0.5.2
############################## WARNING ##############################



KeyError: "28 required columns are missing from the provided dataset to transform using AutoMLPipelineFeatureGenerator. Missing columns: ['Sexo', 'setNumber', 'gameNumber', 'pointNumber', 'serveNumber', 'shotCount', '1º o 2º saque', 'serve_class', 'Lado(1:Iguales;0:Ventaja)', 'V (m/s)', 'timeHIT', 'X1', 'Y1', 'Y1(ABS)', 'Z1 (h)', 'timeNET', 'TimeN-TimeH', 'timeBounce', 'TimeB-TimeH', 'PreVx', 'PreVy', 'PreVz', 'PreV', 'PostVx', 'PostVy', 'PostVz', 'PostV', 'Dif. V1-V2']"

In [2]:
test_df_summary.loc['50%']

Sexo                                         0.000000
setNumber                                    2.000000
gameNumber                                   6.000000
pointNumber                                  4.000000
serveNumber                                  0.000000
shotCount                                    2.000000
1º o 2º saque                                0.000000
Lado(1:Iguales;0:Ventaja)                    1.000000
DIRECCIÓN:1 abierto;2 al cuerpo;3 a la T     2.000000
V (m/s)                                     46.730000
timeHIT                                      1.010000
X1                                         -10.918000
Y1                                          -0.035000
Y1(ABS)                                      1.785000
Z1 (h)                                       2.817000
timeNET                                      1.305000
TimeN-TimeH                                  0.285000
timeBounce                                   1.465000
TimeB-TimeH                 

In [5]:
from ipywidgets import *

def update(serve_class, shotCount, saque, gameNumber, Lado,pointNumber,
           PostVx, Z1_h, V_ms, timeHIT, PreVz):
    d = dict(test_df_summary.loc['50%'])
    # print(d)
    d['serve_class']=serve_class
    d['shotCount']=shotCount
    d['1º o 2º saque'] = saque
    d['gameNumber'] = gameNumber
    d['Lado(1:Iguales;0:Ventaja)'] = Lado
    d['pointNumber'] = pointNumber
    d['PostVx'] = PostVx
    d['Z1 (h)'] = Z1_h
    d['V (m/s)'] = V_ms
    d['timeHIT'] = timeHIT
    d['PreVz'] = PreVz
    df = pd.DataFrame(d, index=[0])
    y_pred = predictor.predict_proba(df, model='NeuralNetTorch_DSTL', as_pandas=True, as_multiclass=True)
    print("Efectividad: ", y_pred.values[0][1])   


interact(
    update, serve_class= widgets.SelectionSlider(options=['NOT RETURNED', 'RETURNED', 'FAULT', 'ACE'],                                            
                                    value='RETURNED'), 
    shotCount= widgets.IntSlider(value=4, min=1, max=12, step=1),
    saque=widgets.IntSlider(value=0, min=0, max=1, step=1),
    gameNumber=widgets.IntSlider(value=6, min=1, max=12, step=1),
    Lado=widgets.IntSlider(value=0, min=0, max=1, step=1),
    pointNumber=widgets.IntSlider(value=4, min=1, max=10, step=1),
    PostVx=widgets.IntSlider(value=8, min=-49, max=45, step=5),
    Z1_h=widgets.FloatSlider(value=2.8, min=1.5, max=4.4, step=0.1),
    V_ms=widgets.IntSlider(value=46, min=21, max=64, step=5),
    timeHIT=widgets.IntSlider(value=1, min=0, max=9, step=1),
    PreVz=widgets.FloatSlider(value=-7.8, min=-10.5, max=-2.8, step=0.1)        
        );

interactive(children=(SelectionSlider(description='serve_class', index=1, options=('NOT RETURNED', 'RETURNED',…

In [ ]:
"""
'serve_class': ['NOT RETURNED', 'RETURNED', 'FAULT', 'ACE'],
'shotCount':[1,2,4,8,12],
'1º o 2º saque':[0,1],
'gameNumber' :[1,3,6,8,12],
'Lado(1:Iguales;0:Ventaja)' :[0,1],
'pointNumber':[1,2,4,5,10],
'PostVx' :[-49, -20, 8, 22, 45],
'Z1 (h)' :[1.5, 2.7, 2.8, 2.9, 4.4],
'V (m/s)' :[21, 42, 46, 51, 64],
'timeHIT' :[0,0.65,1,2,9],
'PreVz' :[-10.5, -8.2,-7.8,-7.4,-2.8]
"""